<a href="https://colab.research.google.com/github/fukagai-takuya/gifu-ai/blob/main/gifu-ai-2025-12-21/FLUX1_schnell_GitClone_Diffusers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Google Drive のマウント (Google Drive 側で予め下記のコマンドを実行していると仮定しています。)

git clone https://github.com/huggingface/diffusers.git

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

2. git clone した diffusers のソースコードを優先して参照するようにします

In [ ]:
import sys
sys.path.insert(0, "/content/drive/MyDrive/diffusers/src")

3. git clone した diffusers のソースコードが優先して参照されることを確認します

In [ ]:
import diffusers
print(diffusers.__file__)

4. diffusers を使用して FLUX.1 [schnell] を実行する際に必要なパッケージをインストールします

In [ ]:
!pip install torch torchvision torchaudio
!pip install transformers accelerate safetensors
!pip install sentencepiece protobuf einops

5. pdb より高機能な Python デバッガ ipdb をインストールします

In [ ]:
!pip install ipdb

6. FLUX.1 [schnell] を使用した Pipeline を用意します

In [ ]:
import torch
from diffusers import FluxPipeline

pipe = FluxPipeline.from_pretrained(
    "black-forest-labs/FLUX.1-schnell",
    torch_dtype=torch.float16
).to("cuda")


In [1]:


image = pipe(
    prompt,
    guidance_scale=0.0,
    num_inference_steps=4,
    max_sequence_length=256,
    generator=torch.Generator("cpu").manual_seed(0)
).images[0]

image.save("Gifu-AI-study-group.png")


NameError: name 'pipe' is not defined

In [ ]:
from PIL import Image
import os

out_dir = "steps"
os.makedirs(out_dir, exist_ok=True)

# GIF 用に画像を溜めるリスト
gif_frames = []

def callback(step: int, timestep: int, latents: torch.FloatTensor):
    with torch.no_grad():
        image = pipe.vae.decode(
            latents / pipe.vae.config.scaling_factor
        ).sample

        image = (image / 2 + 0.5).clamp(0, 1)
        image = image.cpu().permute(0, 2, 3, 1).numpy()

        img = Image.fromarray((image[0] * 255).astype("uint8"))

        # 各 step の PNG 保存（任意）
        img.save(os.path.join(out_dir, f"step_{step:03d}.png"))

        # GIF 用に追加
        gif_frames.append(img)

pipe(
    prompt="a futuristic city at sunset",
    num_inference_steps=8,   # schnell
    callback=callback,
    callback_steps=1
)

# ===== 推論後に GIF を作成 =====
gif_path = "denoising.gif"

gif_frames[0].save(
    gif_path,
    save_all=True,
    append_images=gif_frames[1:],
    duration=200,  # ms / frame
    loop=0
)

print(f"GIF saved to {gif_path}")
